In [1]:
from bs4 import BeautifulSoup
import requests
import urllib
import os
import re
from urllib.request import urlopen

import pandas as pd
import numpy as np

In [2]:
def make_soup(url):
    '''
    INPUT: url string
    OUTPUT: BeautifulSoup object
    '''
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

In [3]:
def grab_body(url):
    '''
    INPUT: blog post url
    OUTPUT: list of sentence strings
    '''
    soup = make_soup(url)
    art = soup.article
    recipe = art.find('div', class_="wprm-recipe-container")
    recipe_start = str(recipe)[:30]
    idx = str(art).index(recipe_start)
    data = str(art)[:idx]
    data = re.sub(r'<.*?>', '', data)
    data = re.sub('\xa0', '', data)
    data = re.sub('\n', ' ', data)
    return data, soup.title.text

In [58]:
def get_bodies_on_page(url):
    df = pd.DataFrame(columns=['title', 'body', 'url'])
    links = get_recipe_links(url)
    for link in links:
        try:
            body, title = grab_body(link)
            A = pd.DataFrame([[title, body, link]], columns=['title', 'body', 'url'])
            df = pd.concat([df,A], axis=0, ignore_index=True)
        except:
            continue
    return df

In [65]:
def get_categories(url):
    soup = make_soup(url)
    cats = soup.find_all('option', class_='level-0')
    cat_names = [cat.text.lower() for cat in cats]
    # remove duplicates
    return cat_names[25:]

In [45]:
def get_recipe_links(url):
    soup = make_soup(url)
    links = [link.get("href") for link in soup.find_all('a', class_='more-link')]
    return links

- for each category
    - for each page
        - get body from each recipe on page

In [74]:
cats = ['pasta', 'breakfast']
final = pd.DataFrame(columns = ['title','body','url'])
for cat in cats:
    url = 'https://thesaltymarshmallow.com/category/{}/'.format(cat)
    i = 0
    
    data = pd.DataFrame(columns=['title','body','url'])
    data = pd.concat([data, get_bodies_on_page(url)], axis=0, ignore_index=True)
    
    url_next = get_next_page(url)
    while i < 37:
        try:
            url = get_next_page(url)
            data = pd.concat([data, get_bodies_on_page(url)], axis=0, ignore_index=True)
            i += 1
        except:
            i+=1
            continue
    final = pd.concat([final, data], axis=0, ignore_index=True)
    

In [75]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 3 columns):
title    46 non-null object
body     46 non-null object
url      46 non-null object
dtypes: object(3)
memory usage: 1.2+ KB


In [57]:
def get_next_page(url):
    soup = make_soup(url)
    next_ = soup.find('li', class_="pagination-next").find("a").get("href")
    return next_

In [55]:
a.find("a").get("href")

'https://thesaltymarshmallow.com/category/dinner/page/2/'

In [79]:
test = pd.read_csv('/Users/clairemason/Galvanize/Capstone/capstone/data/salty_mallow.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
Unnamed: 0    0 non-null object
title         0 non-null object
body          0 non-null object
url           0 non-null object
dtypes: object(4)
memory usage: 0.0+ bytes


In [73]:
get_categories('https://thesaltymarshmallow.com/recipes/')

['appetizers',
 'bars',
 'beverages',
 'breads',
 'breakfast',
 'brownies',
 'cake',
 'casserole',
 'chicken',
 'cookies',
 'cupcakes',
 'dessert',
 'dinner',
 'dips',
 'donuts',
 'eats',
 'grain/gluten free',
 'instant pot recipes',
 'muffins',
 'one pot',
 'pasta',
 'pasta',
 'popular',
 'salad',
 'scones',
 'side dishes',
 'slow cooker',
 'snacks',
 'soup',
 'sweets',
 'uncategorized',
 'video']